# Nanodegree Engenheiro de Machine Learning
## Aprendizagem Supervisionada
## Projeto 2: Construindo um Sistema de Intervenção para Estudantes

Bem-vindo ao segundo projeto do Nanodegree de Machine Learning! Neste Notebook, alguns templates de código já foram fornecidos, e será o seu trabalho implementar funcionalidades necessárias para completar este projeto com êxito. Seções que começam com **'Implementação'** no cabeçalho indicam que o bloco de código que se segue precisará de funcionalidades adicionais que você deve fornecer. Instruções serão providenciadas para cada seção e as especificações para cada implementação estarão marcadas no bloco de código com o comando `'TODO'`. Tenha certeza de ler atentamente todas as instruções!

Além do código implementado, haverá questões relacionadas ao projeto e à implementação que você deve responder. Cada seção em que você tem que responder uma questão será antecedida de um cabeçalho **'Questão X'**. Leia atentamente cada questão e escreva respostas completas nas caixas de texto subsequentes que começam com **'Resposta: '**. O projeto enviado será avaliado baseado nas respostas para cada questão e a implementação que você forneceu.  

>**Nota:** Células de código e Markdown podem ser executadas utilizando o atalho de teclado **Shift + Enter**. Além disso, as células Markdown podem ser editadas, um clique duplo na célula entra no modo de edição.

### Questão 1 - Classificação versus Regressão
*Seu objetivo neste projeto é identificar estudantes que possam precisar de intervenção antecipada antes de serem reprovados. Que tipo de problema de aprendizagem supervisionada é esse: classificação ou regressão? Por quê?*

**Resposta: **

Este é um problema de classificação. Claramente há a necessidade de separar os estudantes em dois grupos: aprovados (ou possível aprovação) e reprovados (ou possível reprovação), caracterizando uma saída discreta, própria de sistemas de classificação.

## Observando os Dados
Execute a célula de código abaixo para carregar as bibliotecas de Python necessárias e os dados sobre os estudantes. Note que a última coluna desse conjunto de dados, `'passed'`, será nosso rótulo alvo (se o aluno foi ou não aprovado). As outras colunas são atributos sobre cada aluno.

In [2]:
# Importar bibliotecas
import numpy as np
import pandas as pd
from time import time
from sklearn.metrics import f1_score

# Ler os dados dos estudantes
student_data = pd.read_csv("student-data.csv")
print "Os dados dos estudantes foram lidos com êxito!"

Os dados dos estudantes foram lidos com êxito!


### Implementação: Observando os Dados
Vamos começar observando o conjunto de dados para determinar quantos são os estudantes sobre os quais temos informações e entender a taxa de graduação entre esses estudantes. Na célula de código abaixo, você vai precisar calcular o seguinte:
- O número total de estudantes, `n_students`.
- O número total de atributos para cada estudante, `n_features`.
- O número de estudantes aprovados, `n_passed`.
- O número de estudantes reprovados, `n_failed`.
- A taxa de graduação da classe, `grad_rate`, em porcentagem (%).


In [3]:
# TODO: Calcule o número de estudante
#n_students = None

# TODO: Calcule o número de atributos
n_students, n_features = student_data.shape

# TODO: Calcule o número de alunos aprovados
n_passed = student_data[student_data.passed == 'yes'].passed.count()

# TODO: Calcule o número de alunos reprovados
n_failed = student_data[student_data.passed == 'no'].passed.count()

# TODO: Calcule a taxa de graduação
grad_rate = float(n_passed)*100/float(n_students)


# Imprima os resultados
print "Número total de estudantes: {}".format(n_students)
print "Número de atributos: {}".format(n_features)
print "Número de estudantes aprovados: {}".format(n_passed)
print "Número de estudantes reprovados: {}".format(n_failed)
print "Taxa de graduação: {:.2f}%".format(grad_rate)

Número total de estudantes: 395
Número de atributos: 31
Número de estudantes aprovados: 265
Número de estudantes reprovados: 130
Taxa de graduação: 67.09%


## Preparando os Dados
Nesta seção, vamos preparara os dados para modelagem, treinamento e teste.

### Identificar atributos e variáveis-alvo
É comum que os dados que você obteve contenham atributos não numéricos. Isso pode ser um problema, dado que a maioria dos algoritmos de machine learning esperam dados númericos para operar cálculos.

Execute a célula de código abaixo para separar os dados dos estudantes em atributos e variáveis-alvo e verificar se algum desses atributos é não numérico.

In [4]:
# Extraia as colunas dos atributo
feature_cols = list(student_data.columns[:-1])

# Extraia a coluna-alvo 'passed'
target_col = student_data.columns[-1] 

# Mostre a lista de colunas
print "Colunas de atributos:\n{}".format(feature_cols)
print "\nColuna-alvo: {}".format(target_col)

# Separe os dados em atributos e variáveis-alvo (X_all e y_all, respectivamente)
X_all = student_data[feature_cols]
y_all = student_data[target_col]

# Mostre os atributos imprimindo as cinco primeiras linhas
print "\nFeature values:"
print X_all.head()

Colunas de atributos:
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']

Coluna-alvo: passed

Feature values:
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

    ...    higher internet  romantic  famrel  freetime goout Dalc Walc health  \
0   ...       yes       no        no       4         3     4    1    1      3   
1   ...    

### Pré-processar Colunas de Atributo

Como você pode ver, há muitas colunas não numéricas que precisam ser convertidas! Muitas delas são simplesmente `yes`/`no`, por exemplo, a coluna `internet`. É razoável converter essas variáveis em valores (binários) `1`/`0`.

Outras colunas, como `Mjob` e `Fjob`, têm mais do que dois valores e são conhecidas como variáveis categóricas. A maneira recomendada de lidar com esse tipo de coluna é criar uma quantidade de colunas proporcional aos possíveis valores (por exemplo, `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc), e assinalar `1` para um deles e `0` para todos os outros.

Essas colunas geradas são por vezes chamadas de _variáveis postiças_ (_dummy variables_), e nós iremos utilizar a função [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) para fazer essa conversão. Execute a célula de código abaixo para executar a rotina de pré-processamento discutida nesta seção.

In [5]:
def preprocess_features(X):
    ''' Pré-processa os dados dos estudantes e converte as variáveis binárias não numéricas em
        variáveis binárias (0/1). Converte variáveis categóricas em variáveis postiças. '''
    
    # Inicialize nova saída DataFrame
    output = pd.DataFrame(index = X.index)

    # Observe os dados em cada coluna de atributos 
    for col, col_data in X.iteritems():
        
        # Se o tipo de dado for não numérico, substitua todos os valores yes/no por 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])

        # Se o tipo de dado for categórico, converta-o para uma variável dummy
        if col_data.dtype == object:
            # Example: 'school' => 'school_GP' and 'school_MS'
            col_data = pd.get_dummies(col_data, prefix = col)  
        
        # Reúna as colunas revisadas
        output = output.join(col_data)
    
    return output

X_all = preprocess_features(X_all)
print "Processed feature columns ({} total features):\n{}".format(len(X_all.columns), list(X_all.columns))

Processed feature columns (48 total features):
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


### Implementação: Divisão dos Dados de Treinamento e Teste
Até agora, nós convertemos todos os atributos _categóricos_ em valores numéricos. Para o próximo passo, vamos dividir os dados (tanto atributos como os rótulos correspondentes) em conjuntos de treinamento e teste. Na célula de código abaixo, você irá precisar implementar o seguinte:
- Embaralhe aleatoriamente os dados (`X_all`, `y_all`) em subconjuntos de treinamento e teste.
  - Utilizar 300 pontos de treinamento (aproxidamente 75%) e 95 pontos de teste (aproximadamente 25%).
  - Estabelecer um `random_state` para as funções que você utiliza, se a opção existir.
  - Armazene os resultados em `X_train`, `X_test`, `y_train` e `y_test`.

In [6]:
# TODO: Importe qualquer funcionalidade adicional de que você possa precisar aqui
from sklearn.model_selection import train_test_split
# TODO: Estabeleça o número de pontos de treinamento
num_train = 300

# Estabeleça o número de pontos de teste
num_test = X_all.shape[0] - num_train

num_random = 123

# TODO: Emabaralhe e distribua o conjunto de dados de acordo com o número de pontos de treinamento e teste abaixo
X_train, X_test, y_train, y_test = train_test_split(X_all,y_all,
                                                    train_size=num_train,
                                                    test_size=num_test,
                                                    random_state=num_random)

# Mostre o resultado da distribuição
print "O conjunto de treinamento tem {} amostras.".format(X_train.shape[0])
print "O conjunto de teste tem {} amostras.".format(X_test.shape[0])

O conjunto de treinamento tem 300 amostras.
O conjunto de teste tem 95 amostras.


## Treinando e Avaliando Modelos
Nesta seção, você irá escolher 3 modelos de aprendizagem supervisionada que sejam apropriados para esse problema e que estejam disponíveis no `scikit-learn`. Primeiro você irá discutir o raciocínio por trás da escolha desses três modelos considerando suas vantagens e desvantagens e o que você sabe sobre os dados. Depois você irá ajustar o modelo a diferentes tamanhos de conjuntos de treinamento (com 100, 200 e 300 pontos) e medir a pontuação F<sub>1</sub>. Você vai precisar preencher três tabelas (uma para cada modelo) que mostrem o tamanho do conjunto de treinamento, o tempo de treinamento, o tempo de previsão e a pontuação F<sub>1</sub> no conjunto de treinamento.

**Os seguintes modelos de aprendizagem supervisionada estão atualmente disponíveis no **[`scikit-learn`](http://scikit-learn.org/stable/supervised_learning.html)** para você escolher:**
- Gaussian Naive Bayes (GaussianNB)
- Árvores de Decisão
- Métodos de agregação (Bagging, AdaBoost, Random Forest, Gradient Boosting)
- K-Nearest Neighbors (KNeighbors)
- Método do gradiente estocástico (SGDC)
- Máquinas de vetores de suporte (SVM)
- Regressão logística

### Questão 2 - Aplicação dos Modelos
*Liste três modelos de aprendizagem supervisionada que são apropriadas para esse problema. Para cada modelo escolhido:*
- Descreva uma aplicação em mundo real na indústria em que o modelo pode ser aplicado. *(Talvez você precise fazer um pouco de pesquisa para responder essa questão – dê as devidas referências!)* 
- Quais são as vantagens do modelo; quando ele tem desempenho melhor? 
- Quais são as desvantagens do modelo, quando ele tem desempenho pior?
- O que faz desse modelo um bom candidato para o problema, considerando o que você sabe sobre os dados?

**Resposta: ** 

Respondendo a última questão em primeiro lugar - o que faz dos modelos escolhidos bons candidatos para o problema - a escolha dos três métodos abaixo foi feita por eliminação, baseando-se nos seguintes critérios:

Necessidade de um algoritmo de classificação com melhor desempenho para classificações binárias - o que descartou os algoritmos de agregação e KNN;

Necessidade de um bom poder de generalização com um conjunto de dados reduzido - o que eliminou o Gaussian Naive Bayes, que possui sua grande utilidade em generalizar conjuntos de dados muito grandes;

Necessidade de um bom poder de generalização sem a necessidade de grande número de ajustes prévios - descartando o Método do gradiente estocástico, cujos resultados variam muito de acordo com a escolha prévia dos seus hiperparâmetros.

Com essas considerações, os modelos restantes foram: Árvores de decisão, Máquinas de vetores de suporte e Regressão logística.
 

**Árvores de decisão:**

As árvores de decisão são normalmente aplicadas em situações onde é interessante se verificar os passos realizados pelo algoritmo. Um bom exemplo é a área médica, onde as árvores de decisão já foram aplicadas no diagnóstico de asma, identificação de indivíduos não aderentes à tratamento anti-hipertensivo e identificação de pacientes propensos ao diabetes, por exemplo.(Fonte: https://periodicos.ufersa.edu.br/index.php/ecop/article/download/7076/6533)  

As vantagens da árvore de de decisão são a facilidade de entendimento do modelo - uma vez que as árvores podem ser visualizadas graficamente; o funcionamento razoável com dados pouco tratados - aceitando inclusive dados em formato não numérico; e o grande desempenho na realização de testes.

As desvantagens são a tendência a se criar árvores complexas - que não generalizam bem os dados; dificuldades de adaptação à mudanças nos dados de treinamento; e um crescimento exponencial do tempo de treinamento em relação ao tamanho do conjunto de dados. 

**Máquinas de vetores de suporte:**

As SVMs são aplicadas principalmente em problemas complexos, não linearmente separáveis. Um exemplo é a sua aplicação para o reconhecimento de rostos, onde a SVM classifica cada parte da imagem como "face" ou "não-face" e cria uma delimitação em torno da face.(Fonte: https://data-flair.training/blogs/applications-of-svm/)

As vantagens são os bons resultados apresentados mesmo em bases de dados com muitas dimensões; os bons resultados obtidos com conjuntos de dados com outliers ou ruidos; e a possibilidade de classificação de conjuntos não linearmente separáveis, através de uso de kernels.

As desvantagens são a dificuldade em se interpretar os resultados do modelo; a incapacidade de gerar estimativas probabilísticas nativamente; e a grande dependência de ajuste nos kernels para se obter bons resultados, o que pode acarretar em um grande uso de recursos computacionais. 


**Regressão logística:**

A regressão logística é muito utilizada em áreas onde é interessante se verificar ou explicar o impacto de uma determinada variável no resultado. Um exemplo é o uso na área de econometria, para explicar uma variável discreta, como por exemplo as intenções de voto.(Fonte: https://pt.wikipedia.org/wiki/Regress%C3%A3o_log%C3%ADstica)

As vantagens da Regressão logística são a sua simplicidade intrínseca, que evita sobreajustes; a disponibilidade de uma análise probabilística dos seus resultados; e a possibilidade de uma fácil análise do impacto de uma variável sobre o resultado.

As desvantagens são a mudança significativa nos resultados de conjuntos de dados com outliers; a necessidade de ajustes prévios do conjunto de dados, principalmente para a verificação de variáveis com alta correlação entre si; e um mau desempenho em problemas não linearmente separáveis, como a porta XOR, por exemplo, ; 

### Configuração
Execute a célula de código abaixo para inicializar três funções de ajuda que você pode utilizar para treinar e testar os três modelos de aprendizagem supervisionada que você escolheu acima. As funções são as seguintes:
- `train_classifier` - recebe como parâmetro um classificador e dados de treinamento e ajusta o classificador aos dados.
- `predict_labels` - recebe como parâmetro um classificador ajustado, atributos e rótulo alvo e faz estimativas utilizando a pontuação do F<sub>1</sub>.
- `train_predict` - recebe como entrada um classificador, e dados de treinamento e teste, e executa `train_clasifier` e `predict_labels`.
 - Essa função vai dar a pontuação F<sub>1</sub> tanto para os dados de treinamento como para os de teste, separadamente.

In [7]:
def train_classifier(clf, X_train, y_train):
    ''' Ajusta um classificador para os dados de treinamento. '''
    
    # Inicia o relógio, treina o classificador e, então, para o relógio
    start = time()
    clf.fit(X_train, y_train)
    end = time()
    
    # Imprime os resultados
    print "O modelo foi treinado em {:.4f} segundos".format(end - start)

    
def predict_labels(clf, features, target):
    ''' Faz uma estimativa utilizando um classificador ajustado baseado na pontuação F1. '''
    
    # Inicia o relógio, faz estimativas e, então, o relógio para
    start = time()
    y_pred = clf.predict(features)
    end = time()
    
    # Imprime os resultados de retorno
    print "As previsões foram feitas em {:.4f} segundos.".format(end - start)
    return f1_score(target.values, y_pred, pos_label='yes')


def train_predict(clf, X_train, y_train, X_test, y_test):
    ''' Treina e faz estimativas utilizando um classificador baseado na pontuação do F1. '''
    
    # Indica o tamanho do classificador e do conjunto de treinamento
    print "Treinando um {} com {} pontos de treinamento. . .".format(clf.__class__.__name__, len(X_train))
    
    # Treina o classificador
    train_classifier(clf, X_train, y_train)
    
    # Imprime os resultados das estimativas de ambos treinamento e teste
    print "Pontuação F1 para o conjunto de treino: {:.4f}.".format(predict_labels(clf, X_train, y_train))
    print "Pontuação F1 para o conjunto de teste: {:.4f}.".format(predict_labels(clf, X_test, y_test))

### Implementação: Métricas de Desempenho do Modelo
Com as funções acima, você vai importar os três modelos de aprendizagem supervisionada de sua escolha e executar a função `train_prediction` para cada um deles. Lembre-se de que você vai precisar treinar e usar cada classificador para três diferentes tamanhos de conjuntos de treinamentos: 100, 200 e 300 pontos. Então você deve ter 9 saídas diferentes abaixo – 3 para cada modelo utilizando cada tamanho de conjunto de treinamento. Na célula de código a seguir, você deve implementar o seguinte:
- Importe os três modelos de aprendizagem supervisionada que você escolheu na seção anterior.
- Inicialize os três modelos e armazene eles em `clf_A`, `clf_B` e `clf_C`.
 - Defina um `random_state` para cada modelo, se a opção existir.
 - **Nota:** Utilize as configurações padrão para cada modelo – você vai calibrar um modelo específico em uma seção posterior.
- Crie diferentes tamanhos de conjuntos de treinamento para treinar cada modelo.
 - *Não embaralhe e distribua novamente os dados! Os novos pontos de treinamento devem ser tirados de `X_train` e `y_train`.*
- Treine cada modelo com cada tamanho de conjunto de treinamento e faça estimativas com o conjunto de teste (9 vezes no total).  
**Nota:** Três tabelas são fornecidas depois da célula de código a seguir, nas quais você deve anotar seus resultados.

In [8]:
# TODO: Importe os três modelos de aprendizagem supervisionada do sklearn
from sklearn import tree
from sklearn import svm
from sklearn import linear_model

# TODO: Inicialize os três modelos
clf_A = tree.DecisionTreeClassifier()
clf_B = svm.SVC()
clf_C = linear_model.LogisticRegression()

# TODO: Configure os tamanho dos conjuntos de treinamento
X_train_100, X_test_100, y_train_100, y_test_100 = train_test_split(X_all,y_all,
                                            train_size=100,
                                            test_size=X_all.shape[0]-100,
                                            random_state=num_random)

X_train_200, X_test_200, y_train_200, y_test_200 = train_test_split(X_all,y_all,
                                            train_size=200,
                                            test_size=X_all.shape[0]-200,
                                            random_state=num_random)

X_train_300, X_test_300, y_train_300, y_test_300 = train_test_split(X_all,y_all,
                                            train_size=300,
                                            test_size=X_all.shape[0]-300,
                                            random_state=num_random)

# TODO: Executar a função 'train_predict' para cada classificador e cada tamanho de conjunto de treinamento
# train_predict(clf, X_train, y_train, X_test, y_test)
train_predict(clf_A,X_train_100,y_train_100,X_test_100,y_test_100)
train_predict(clf_A,X_train_200,y_train_200,X_test_200,y_test_200)
train_predict(clf_A,X_train_300,y_train_300,X_test_300,y_test_300)
train_predict(clf_B,X_train_100,y_train_100,X_test_100,y_test_100)
train_predict(clf_B,X_train_200,y_train_200,X_test_200,y_test_200)
train_predict(clf_B,X_train_300,y_train_300,X_test_300,y_test_300)
train_predict(clf_C,X_train_100,y_train_100,X_test_100,y_test_100)
train_predict(clf_C,X_train_200,y_train_200,X_test_200,y_test_200)
train_predict(clf_C,X_train_300,y_train_300,X_test_300,y_test_300)

Treinando um DecisionTreeClassifier com 100 pontos de treinamento. . .
O modelo foi treinado em 0.0040 segundos
As previsões foram feitas em 0.0010 segundos.
Pontuação F1 para o conjunto de treino: 1.0000.
As previsões foram feitas em 0.0000 segundos.
Pontuação F1 para o conjunto de teste: 0.7444.
Treinando um DecisionTreeClassifier com 200 pontos de treinamento. . .
O modelo foi treinado em 0.0010 segundos
As previsões foram feitas em 0.0000 segundos.
Pontuação F1 para o conjunto de treino: 1.0000.
As previsões foram feitas em 0.0000 segundos.
Pontuação F1 para o conjunto de teste: 0.7059.
Treinando um DecisionTreeClassifier com 300 pontos de treinamento. . .
O modelo foi treinado em 0.0010 segundos
As previsões foram feitas em 0.0010 segundos.
Pontuação F1 para o conjunto de treino: 1.0000.
As previsões foram feitas em 0.0000 segundos.
Pontuação F1 para o conjunto de teste: 0.7612.
Treinando um SVC com 100 pontos de treinamento. . .
O modelo foi treinado em 0.0030 segundos
As previsõ

### Resultados em tabelas
Edite a célula abaixo e veja como a tabela pode ser desenhada em [Markdown](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet#tables). Você deve salvar seus resultados abaixo nas tabelas fornecidas.

** Classificador 1 - Árvores de Decisão ** 

| Tamanho do Conjunto de Treinamento | Tempo de Treinamento | Tempo de Estimativa (teste) | Pontuação F1 (treinamento) | Pontuação F1 (teste) |
| :--------------------------------: | :------------------: | :-------------------------: | :------------------------: | :------------------: |
| 100                                | 0.004               | 0.001                      | 1                     | 0.7525               |
| 200                                |       0.002        |      0.001                       |   1                         | 0.7168                     |
| 300                                |  0.003                    |  0.001                           |      1                      |        0.7442      |

** Classificador 2 - Máquinas de Vetor de Suporte**  

| Tamanho do Conjunto de Treinamento | Tempo de Treinamento | Tempo de Estimativa (teste) | Pontuação F1 (treinamento) | Pontuação F1 (teste) |
| :--------------------------------: | :------------------: | :-------------------------: | :------------------------: | :------------------: |
| 100                                | 0.002                     | 0.002                             |0.9020                            | 0.8101                     |
| 200                                | 0.005        | 0.003                             | 0.8553                           | 0.8243                      |
| 300                                |0.007                      | 0.002                            |0.8515                            |        0.8790       |

** Classificador 3 - Regressão Logística**  

| Tamanho do Conjunto de Treinamento | Tempo de Treinamento | Tempo de Estimativa (teste) | Pontuação F1 (treinamento) | Pontuação F1 (teste) |
| :--------------------------------: | :------------------: | :-------------------------: | :------------------------: | :------------------: |
| 100                                | 0.002                     |0.001                             |0,9231                            |0.7447                      |
| 200                                | 0.002                      | 0.001                            | 0.8633                            |0.7687                      |
| 300                                | 0.003                     | 0.002                            | 0.8326                           | 0.8630                     |

## Escolhendo o Melhor Modelo
Nesta seção final, você irá escolher dos três modelos de aprendizagem supervisionada o *melhor* para utilizar os dados dos estudantes. Você então executará um busca em matriz otimizada para o modelo em todo o conjunto de treinamento (`X_train` e `y_train`) ao calibrar pelo menos um parâmetro, melhorando em comparação a pontuação F<sub>1</sub> do modelo não calibrado. 

### Questão 3 - Escolhendo o Melhor Modelo
*Baseando-se nos experimentos que você executou até agora, explique em um ou dois parágrafos ao conselho de supervisores qual modelo que você escolheu como o melhor. Qual modelo é o mais apropriado baseado nos dados disponíveis, recursos limitados, custo e desempenho?*

**Resposta: **

O melhor modelo foi o Support Vector Machine - ele foi o que conseguiu as maiores pontuações F1 para os conjuntos de testes, em todas as variações.

Os tempos de treinamento e teste foram, em média, o dobro dos outros dois algoritmos, mas sem gerar impacto significativo neste conjunto de dados e se justificando pelo melhor resultado final. Porém, em um conjunto de dados com uma magnitude maior, esta diferença de tempo de treinamento e testes poderia gerar grande impacto e justificaria o uso de outro algoritmo.  

### Questão 4 – O Modelo para um Leigo
*Em um ou dois parágrafos, explique para o conselho de supervisores, utilizando termos leigos, como o modelo final escolhido deve trabalhar. Tenha certeza que você esteja descrevendo as melhores qualidades do modelo, por exemplo, como o modelo é treinado e como ele faz uma estimativa. Evite jargões técnicos ou matemáticos, como descrever equações ou discutir a implementação do algoritmo.*

**Resposta: **

As Máquinas de Vetores de Suporte são algortimos que realizam a separação linear de um conjunto de dados - no caso dos nossos dados, a separação entre quem está aprovado ou reprovado. A separação linear consiste, grosseiramente falando, na divisão dos dados em dois conjuntos, separados um uma "linha".

Este algoritmo procura alcançar a maior margem possível entre os dados - o que quer dizer que ele procura separar os dados com a melhor margem de segurança possível, o que provoca uma melhor capacidade de predição de aprovações ou reprovações futuras.

Além disso, caso o conjunto de dados não possa ser, inicialmente, linearmente separável - caso não se possa passar uma "linha" que separe os conjuntos - ainda é possível usar o algoritmo, desde que se aplique alguma função aos dados de entrada antes da análise. Estas funções são técnicamente chamadas de "kernels" e são capazes de permitir as máquinas de vetores de suporte realizar treinamento e previsões em uma grande variedade de conjuntos de dados.

### Implementação: Calibrando o Modelo (_Tuning_)
Calibre o modelo escolhido. Utilize busca em matriz (`GridSearchCV`) com, pelo menos, um parâmetro importante calibrado com, pelo menos, 3 valores diferentes. Você vai precisar utilizar todo o conjunto de treinamento para isso. Na célula de código abaixo, você deve implementar o seguinte:
- Importe [`sklearn.grid_search.gridSearchCV`](http://scikit-learn.org/stable/modules/generated/sklearn.grid_search.GridSearchCV.html) e [`sklearn.metrics.make_scorer`](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.make_scorer.html).
- Crie o dicionário de parâmetros que você deseja calibrar para o modelo escolhido.
 - Examplo: `parameters = {'parameter' : [list of values]}`.
- Inicialize o classificador que você escolheu e armazene-o em `clf`.
- Crie a função de pontuação F<sub>1</sub> utilizando `make_scorer` e armazene-o em `f1_scorer`.
 - Estabeleça o parâmetro `pos_label` para o valor correto!
- Execute uma busca em matriz no classificador `clf` utilizando o `f1_scorer` como método de pontuação e armazene-o em `grid_obj`.
- Treine o objeto de busca em matriz com os dados de treinamento (`X_train`, `y_train`) e armazene-o em `grid_obj`.

In [11]:
# TODO: Importe 'GridSearchCV' e 'make_scorer'

from sklearn.grid_search import GridSearchCV
from sklearn.metrics import make_scorer
# TODO: Crie a lista de parâmetros que você gostaria de calibrar
parameters = {'kernel':['poly','sigmoid','rbf','linear'],
             'degree':range(1,10),
             'gamma':[x*0.001 for x in range(0,10)],
             'probability':[True,False],
             'shrinking':[True,False]}             

# TODO: Inicialize o classificador
clf = svm.SVC()

# TODO: Faça uma função de pontuação f1 utilizando 'make_scorer'
from sklearn.metrics import f1_score
f1_scorer = make_scorer(f1_score, average='micro')

# TODO: Execute uma busca em matriz no classificador utilizando o f1_scorer como método de pontuação
grid_obj = GridSearchCV(clf,parameters,scoring=f1_scorer)

# TODO: Ajuste o objeto de busca em matriz para o treinamento de dados e encontre os parâmetros ótimos
grid_obj.fit(X_train_300,y_train_300)

# Get the estimator
clf = grid_obj.best_estimator_

# Reporte a pontuação final F1 para treinamento e teste depois de calibrar os parâmetrosprint "Tuned model has a training F1 score of {:.4f}.".format(predict_labels(clf, X_train, y_train))
print "O modelo calibrado tem F1 de {:.4f} no conjunto de treinamento.".format(predict_labels(clf, X_train, y_train))
print "O modelo calibrado tem F1 de {:.4f} no conjunto de teste.".format(predict_labels(clf, X_test, y_test))

As previsões foram feitas em 0.0040 segundos.
O modelo calibrado tem F1 de 0.8274 no conjunto de treinamento.
As previsões foram feitas em 0.0020 segundos.
O modelo calibrado tem F1 de 0.8889 no conjunto de teste.


### Questão 5 - Pontuação F<sub>1</sub> Final
*Qual é a pontuação F<sub>1</sub> do modelo final para treinamento e teste? Como ele se compara ao modelo que não foi calibrado?*

**Resposta: **

A pontuação F1 de treinamento foi 0.8274 e de teste 0.8889.

A pontuação do treinamento foi inferior aos 0.8515 anteriores, mas superou a pontuação de teste, que era 0.8790.





> **Nota**: Uma vez que você completou todas as implementações de código e respondeu todas as questões acima com êxito, você pode finalizar seu trabalho exportando o iPython Nothebook como um document HTML. Você pode fazer isso utilizando o menu acima e navegando para  
**File -> Download as -> HTML (.html)**. Inclua a documentação final junto com o notebook para o envio do seu projeto.